# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [1]:
# import xlwings as xw
# from xlwings.constants import AutoFillType
# import numpy as np
# wb = xw.Book('./07_excel_data/себестоимостьА_в1.xlsx')
# sheet = wb.sheets["Рецептура"]




In [2]:
# cons = sheet.range("G7:O10").options(np.array).value
# cons

In [3]:
# unit = sheet.range("G14:O14").options(np.array).value
# unit

In [4]:
# cost_price = np.nan_to_num(cons * unit).sum(axis=1)
# cost_price

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [5]:
# sheet.range('T7:T10').options(transpose=True).value = cost_price

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [6]:
# sheet.range("T6").value = "Себестоимость" # значение в позиции T6

In [7]:
# sheet.range("T4:T6").api.merge()
# sheet.range("T7:T10").color = (255, 0, 0)

In [8]:
# sheet.range("V7").formula = "=SUMPRODUCT(G7:O7, $G$14:$O$14)"


4. Выполнить 3 с помощью "протягиваемых" формул.

In [9]:
# sheet.range("V7").api.autofill(destination=sheet.range("V7:V10").api,
#                                type=AutoFillType.xlFillDefault)


## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` и `reviews_sample.csv` (__ЛР2__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [10]:
import xlwings as xw
import pandas as pd
import numpy as np
from xlwings.constants import *

recipes = pd.read_csv("./07_excel_data/recipes_sample.csv", sep=",", parse_dates=['submitted'])
recipes = recipes.drop(columns=["n_steps", "contributor_id"])
recipes.head()

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN


In [11]:
review = pd.read_csv("./07_excel_data/reviews_sample.csv", sep=",", parse_dates=['date'])
review.head()

,id,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [12]:
recipes_5per = recipes.sample(n=int(len(recipes) * 0.05), replace=False, axis=0)
review_5per = review.sample(n=int(len(review) * 0.05), replace=False, axis=0)

writer = pd.ExcelWriter('./07_excel_data/out/recipes.xlsx')
recipes_5per.to_excel(writer, sheet_name='Рецепты') 
review_5per.to_excel(writer, sheet_name='Отзывы') 
writer.save()

In [13]:
# recipes_5per

In [14]:
# review_5per

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [15]:
recipes_5per["seconds_assign"] = recipes_5per["minutes"] * 60
recipes_5per["seconds_assign"].head()

21061    15600
1222      1200
15821     1200
7082       600
21964      300
Name: seconds_assign, dtype: int64

In [16]:
wb = xw.Book('./07_excel_data/out/recipes.xlsx')
sheet = wb.sheets["Рецепты"]

In [17]:
sheet.range("H1").value = "seconds_assign"
sheet.range("H1").api.font_object.font_style.set('bold')
sheet.range("H2:H1501").options(transpose=True).value = recipes_5per["seconds_assign"].to_numpy()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [18]:
sheet.range("I1").value = "seconds_formula"
sheet.range("I1").api.font_object.font_style.set('bold')
sheet.range("I2").formula = "=D2 * 60"
sheet.range("I2").api.autofill(destination=sheet.range("I2:I1501").api,
                               type=AutoFillType.xlFillDefault)

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [19]:
sheet.range("J1").value = "n_reviews"
sheet.range("J1").api.font_object.font_style.set('bold')


#TODO
C_values = sheet.range("C2:C1501").value
# for i in range(len(C_values)):
    

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [20]:
# c : -4108; r : -4152; l : -4131;

# sheet['H1:J1'].api.HorizontalAlignment = HAlign.xlHAlignCenter

table = xw.Range("A1:I1501")
table.columns[1].column_width = 45
for i in range(4, 9):
    table.columns[i].column_width = 20


7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [21]:
values = sheet.range("D2:D1501").value

for i in range(len(values)):
    if values[i] <= 5:
        sheet.range(f"D{i+2}").color = (0, 255, 0)
    elif 5 < values[i] <= 10:
        sheet.range(f"D{i+2}").color = (255, 255, 0)
    elif values[i] > 10:
        sheet.range(f"D{i+2}").color = (255, 0, 0)


8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [22]:
#TODO